In [1]:
import argparse
import concurrent.futures
import logging
import os
from queue import Queue
from threading import Thread
from typing import Dict, List
from PIL import Image
import cv2
import numpy as np

import face_tool
from face_model import FaceModel
from fps_metric import FPS
from scrfd import SCRFD
from utils import check_in_out, get_data, maybe_create_folder, open_cam_usb
from web_cam import WebcamVideoStream
import glob

/Users/dominhtuan/anaconda3/lib/python3.11/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:69: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


In [1]:
!pip install --upgrade streamlit

DEPRECATION: Loading egg at /Users/dominhtuan/anaconda3/lib/python3.11/site-packages/dlib-19.24.99-py3.11-macosx-11.1-arm64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 20.0 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: streamlit
    Found existing installation: streamlit 1.29.0
    Uninstalling streamlit-1.29.0:
      Successfully uninstalled streamlit-1.29.0

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [2]:
face_detector = SCRFD(model_file='/Users/dominhtuan/Downloads/AnNinhSoiChieu/scrfd_10g_bnkps.onnx')
face_model = FaceModel(onnx_model_path='/Users/dominhtuan/Downloads/AnNinhSoiChieu/webface_r50.onnx')

width = 1280
height = 720
WINDOW_NAME = 'CameraCheckIn'

/Users/dominhtuan/anaconda3/lib/python3.11/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:69: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


In [3]:
import face_tool
def get_vector_img(image,theshold_w=0.4,theshold_h=0.2):
    face_boxes, kpss = face_detector.detect(image,theshold_w =theshold_w,theshold_h=theshold_h,thresh=0.2)
    if face_boxes is not None:
        face_boxes = face_boxes.astype(np.int32)
        face_boxes = face_boxes[:, :4]
        face_boxes = np.maximum(face_boxes, 0)
    face_list = []
    if face_boxes is not None:
        for (x1, y1, x2, y2) in face_boxes:
            face_list.append((x1, y1, x2, y2))
        face_encoding = face_tool.face_encoding(image=image, kpss=kpss)
    
        return face_list,face_encoding
    return None,None

In [4]:
image_cc = cv2.imread("/Users/dominhtuan/Downloads/anhcccdtinh.jpg")
face_list,face_encoding_cc = get_vector_img(image_cc,theshold_w=0.05,theshold_h=0.05)

In [5]:
def open_window(width, height):
    cv2.namedWindow(WINDOW_NAME, cv2.WINDOW_NORMAL)
    cv2.resizeWindow(WINDOW_NAME, width, height)
    cv2.moveWindow(WINDOW_NAME, 0, 0)
    cv2.setWindowTitle(WINDOW_NAME, 'Camera Demo for Jetson Nano')

def recognize_face(                                                                    
    image: np.ndarray,                                                                 
    kpss: np.ndarray,                                                                  
    known_faces: List[np.ndarray],                                                                                                                       
) -> List[str]:                                                                        
    face_encodings = face_tool.face_encoding(image=image, kpss=kpss)                   
    face_names = []                                                                    
                                                                                       
    # Face recognition                                                                 
    for face_encoding in face_encodings:                                               
        matches = face_tool.compare_faces(known_faces, face_encoding, tolerance=0.6)   
        face_distances = face_tool.face_distance(known_faces, face_encoding)                                   
        print(matches)
        print(face_distances)
        best_idx = np.argmin(face_distances)                                           
        name = "Unknown"                                                               
        if matches[best_idx]:                                                          
            name = "MR.KI"                                                
                                                                      
        if name == "Unknown":                                                          
            face_names.append("Unknown")                                               
        else:                                                                                    
            face_names.append(name)                                                                                   
    return face_names                                                                  

def worker(input_q: Queue, output_q: Queue,known_faces):
    while True:
        frame, kpss = input_q.get()
        names = recognize_face(     
                image=frame,            
                kpss=kpss,              
                known_faces=known_faces
            )                           
        output = {"names": names}
        output_q.put(output)

def start_app(video_capture):
    frame_index = 0
    face_step = 30
    detect_step = 15
    reset_step = 120

    input_q = Queue(2)  # fps is better if queue is higher but then more lags
    output_q = Queue()
    for i in range(1):
        t = Thread(target=worker, args=(input_q, output_q,face_encoding_cc))
        t.daemon = True
        t.start()

    fps = FPS().start()
    while True:
        _, frame = video_capture.read()
        flag = False
        frame = cv2.flip(frame, 1)
        face_list = []
        name ="Unknown"
        if frame_index % detect_step == 0:
            face_boxes, kpss = face_detector.detect(frame)
            if face_boxes is not None:
                face_boxes = face_boxes.astype(np.int32)
                face_boxes = face_boxes[:, :4]
                face_boxes = np.maximum(face_boxes, 0)
                for (x1, y1, x2, y2) in face_boxes:
                    face_list.append((x1, y1, x2, y2))
                    img_save = frame[y1:y2,x1:x2]
                    number_of_images = len(glob.glob('/Users/dominhtuan/Downloads/AnNinhSoiChieu/image_save/*'))
                    if  number_of_images < 4:
                        cv2.imwrite(f'image_save/output_image_{number_of_images +1}.jpg', img_save)
            if kpss is not None and frame_index % face_step == 0:
                if not input_q.full():
                    print("1111")
                    input_q.put((frame, kpss))
        print(output_q.empty())
        if output_q.empty():
            pass  # fill up queue
        else:
            data = output_q.get()
            face_names = data['names']
            name = face_names[-1]
        if frame_index % (reset_step * 10) == 0:
            checked_name = {}
            frame_index = 0
    
        font = cv2.FONT_HERSHEY_COMPLEX

        for (x1, y1, x2, y2) in face_list:
            top = y1
            bottom = y2
            left = x1
            right = x2
            if name != "Unknown":                                                                                                                  
                cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)                              
                cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 255, 0), cv2.FILLED)             
                cv2.putText(frame, "{}".format(name), (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)    
            else:                                                                                               
                cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)                              
                cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)             
                if name == "Unknown":                                                                           
                    cv2.putText(frame, "Unknown", (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)        
                else:                                                                                           
                    cv2.putText(frame, "{}".format(name), (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)


        cv2.imshow(WINDOW_NAME, frame)
        cv2.moveWindow(WINDOW_NAME, 0, 0)
        fps.update()

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    fps.stop()

In [6]:
video_capture = open_cam_usb(1)
open_window(width, height)

start_app(video_capture)

video_capture.release()
cv2.destroyAllWindows()

True
1111
True
[True]
[0.468]
1111
False
[True]
[0.444]
1111
False
[True]
[0.489]
1111
False
[True]
[0.426]
1111
False
[True]
[0.447]
1111
False
[True]
[0.404]
1111
False
[True]
[0.423]
1111
False
[True]
[0.427]
1111
False
[True]
[0.421]
1111
False
[True]
[0.485]
1111
False
[True]
[0.484]
1111
False
[True]
[0.448]
1111
False
[True]
[0.457]
1111
False
[True]
[0.476]
1111
False
[True]
[0.451]
1111
False
[True]
[0.423]
1111
False
[True]
[0.456]
1111
False
[True]
[0.452]
1111
False
[True]
[0.432]
1111
False
[True]
[0.453]
1111
False
[True]
[0.447]
1111
False
[True]
[0.456]
1111
False
[True]
[0.466]
1111
False
[True]
[0.449]
1111
False
[True]
[0.436]
1111
False
[True]
[0.432]
1111
False
[True]
[0.445]
1111
False
[True]
[0.461]
1111
False
[True]
[0.451]
1111
False
[True]
[0.452]
1111
False
[True]
[0.432]
1111
False
[True]
[0.438]
1111
False
[True]
[0.437]
1111
False
[True]
[0.454]
1111
False
[True]
[0.442]
1111
False
[True]
[0.444]
1111
False
[True]
[0.46]
1111
False
[True]
[0.45]
1111
False